In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
pd.set_option('display.max_columns',None)



from pandas.api.types import CategoricalDtype

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly
from tqdm import tqdm

sns.set(style='whitegrid')
plt.rcParams['figure.figsize']=(15,5)
%config InlineBackend.figure_format = 'png'
%matplotlib inline

Загружаем данные в датафреймы 

In [2]:
df_ost = pd.read_pickle("./df_ost.pkl")
df_category = pd.read_pickle("./df_category.pkl")
df_worst_milk_sales_bad_shops = pd.read_pickle("./df_worst_milk_sales_bad_shops.pkl")
df_best_milk_sales_good_shops = pd.read_pickle("./df_best_milk_sales_good_shops.pkl")

In [3]:
#Исправляем табличку 
df_category.columns = list(map(''.join, df_category.columns.values))

In [4]:
df_category

,id_group,name_gr,id_group2,name_gr2,id_tov,name_tov,date_ttcount
0,11,Прочее,100,ХозТовары,942,Комус Ёмкость РК-19,6
1,11,Прочее,100,ХозТовары,1260,Кассовая лента ВкусВилл (широкая),26
2,11,Прочее,100,ХозТовары,1269,Зубочистки (1000шт),2
3,11,Прочее,100,ХозТовары,1271,Стаканчики (100шт),3
4,11,Прочее,100,ХозТовары,1272,Скотч,5
...,...,...,...,...,...,...,...
6640,10966,Чай и кофе. Проекты,10662,Сырье для Кофе с собой,30085,"Кофе зерновой BLEND #5, 1кг",71
6641,10966,Чай и кофе. Проекты,10662,Сырье для Кофе с собой,30572,Кофе Флэт Уайт 0.2,6
6642,10966,Чай и кофе. Проекты,10662,Сырье для Кофе с собой,39254,"Какао-напиток ""Choco 01 Rich Dark"", 1кг",141
6643,10966,Чай и кофе. Проекты,10662,Сырье для Кофе с собой,39255,"Кофе зерновой ""SOLD OUT"", 1 кг",141


In [5]:
df_category_milk = df_category[['name_gr2','name_tov','date_ttcount']][(df_category['name_gr'] =='Молочная Продукция') & (df_category['date_ttcount'] > 3000)]

In [6]:

lis_good_shops = df_best_milk_sales_good_shops.index.values.tolist()
lis_bad_shops = df_worst_milk_sales_bad_shops.index.values.tolist()

In [7]:
milk_items = df_category_milk['name_tov'].unique().tolist()

In [12]:
def OOS2(cat, group_type):
    bad_shop_oos_stock = df_ost[(df_ost[group_type] == cat) 
    & (df_ost['evening'].isna()) 
    & (df_ost['id_tt'].isin(lis_bad_shops))]

    good_shop_oos_stock = df_ost[(df_ost[group_type] == cat) 
    & (df_ost['evening'].isna()) 
    & (df_ost['id_tt'].isin(lis_good_shops))]

    OOS_in_good_shops  = len(good_shop_oos_stock.index)//15
    OOS_in_bad_shops  = len(bad_shop_oos_stock.index)//19
    OOS_percent = 1- len(good_shop_oos_stock.index)/(len(bad_shop_oos_stock.index)+1)

    return OOS_in_good_shops, OOS_in_bad_shops , OOS_percent

In [9]:
df_OOS2 = pd.DataFrame(columns = ['item','OOS_in_good_shops','OOS_in_bad_shops','OOS_percent'])

In [13]:
print(OOS2('Молочная Продукция', 'name_gr'))

(1288, 2164, 0.5298981105464096)


In [115]:
for gr in tqdm(milk_items):
    OOS_in_good_shops ,  OOS_in_bad_shops , OOS_percent = OOS2(gr ,'name_tov')
    new_row = {'item':gr, 'OOS_in_good_shops':OOS_in_good_shops, 'OOS_in_bad_shops':OOS_in_bad_shops, 'OOS_percent':OOS_percent}
    df_OOS2 = pd.concat([df_OOS2,pd.DataFrame([new_row])], axis=0, ignore_index=True)

100%|██████████| 127/127 [04:17<00:00,  2.03s/it]


In [122]:
df_OOS2

,item,OOS_in_good_shops,OOS_in_bad_shops,OOS_percent,name_tov,id_tov
0,Молоко цельное,86,140,0.51773,Молоко цельное,61.0
1,"Молоко 3,2%",182,256,0.439689,"Молоко 3,2%",173.0
2,"Молоко 2,5% в бутылке, 450 мл",121,192,0.502591,"Молоко 2,5% в бутылке, 450 мл",15433.0
3,"Молоко 3,2% в бутылке",154,240,0.493776,"Молоко 3,2% в бутылке",17525.0
4,Молоко цельное в бутылке,98,132,0.413534,Молоко цельное в бутылке,17736.0
...,...,...,...,...,...,...
249,"Кефир 3,2% с крышкой",86,104,0.352381,NaN,NaN
250,"Кефир 3,2%, 290 г",162,266,0.520599,NaN,NaN
251,"Кефир 2,5%, 290 г",238,320,0.41433,NaN,NaN
252,"Молоко безлактозное 3,5-4,5%",101,242,0.670782,NaN,NaN


In [107]:
df_OOS2.sort_values('OOS_percent').to_excel('df_OOS2.xlsx')

In [121]:
df_OOS2


,item,OOS_in_good_shops,OOS_in_bad_shops,OOS_percent,name_tov,id_tov
0,Молоко цельное,86,140,0.51773,Молоко цельное,61.0
1,"Молоко 3,2%",182,256,0.439689,"Молоко 3,2%",173.0
2,"Молоко 2,5% в бутылке, 450 мл",121,192,0.502591,"Молоко 2,5% в бутылке, 450 мл",15433.0
3,"Молоко 3,2% в бутылке",154,240,0.493776,"Молоко 3,2% в бутылке",17525.0
4,Молоко цельное в бутылке,98,132,0.413534,Молоко цельное в бутылке,17736.0
...,...,...,...,...,...,...
249,"Кефир 3,2% с крышкой",86,104,0.352381,NaN,NaN
250,"Кефир 3,2%, 290 г",162,266,0.520599,NaN,NaN
251,"Кефир 2,5%, 290 г",238,320,0.41433,NaN,NaN
252,"Молоко безлактозное 3,5-4,5%",101,242,0.670782,NaN,NaN


In [119]:
df_OOS3 = df_OOS2.merge(df_category[['name_tov','id_tov']], left_on='item', right_on='name_tov', how='left') 


In [123]:
del df_OOS3['name_tov_x']
del df_OOS3['name_tov_y']


In [125]:
del df_OOS3['id_tov_x']

In [128]:
df_OOS3 = df_OOS3.drop_duplicates(subset=["id_tov_y"])

In [129]:
df_OOS3.sort_values('OOS_percent')

,item,OOS_in_good_shops,OOS_in_bad_shops,OOS_percent,id_tov_y
69,"Масло крестьянское 72,5%, 200 г",30,16,-0.411765,23071
68,"Масло сливочное 82,5%, 200 г",40,30,-0.032258,16306
24,Сметана 25%,78,70,0.126761,52
33,Сливки питьевые 10% порционные,22,20,0.142857,19844
49,"Коктейль детский молочный с бананом 2,5%",63,60,0.180328,21435
...,...,...,...,...,...
118,"Кефир 3,2% в бутылке",93,262,0.718631,18271
30,Сметана 15% Вологодская,98,286,0.728223,58271
26,Сметана 15%,53,160,0.73913,14526
15,"Творог мягкий ""Вишня"" 6%",58,228,0.799127,18120


In [135]:
milk_items_by_id = df_OOS3['id_tov_y'].to_list()

In [130]:
df_checks_short = pd.read_pickle("./df_checks_short.pkl")

In [131]:
df_checks_short.head()

,CheckUID,id_tt_cl,id_tov_cl,Quantity,week,BaseSum,hour
0,32C04617-1DA6-EC11-9064-005056A7A8DF,16061,25528,1.0,11,422.73,20
1,7CC04B95-79AA-EC11-9064-005056A7A8DF,11846,1609,2.0,12,67.62,10
2,7BC04CF0-8AA6-EC11-9064-005056A7A8DF,11160,49220,1.0,11,124.17,10
3,75C04F15-4C9A-EC11-9064-005056A7A8DF,14258,55220,1.0,9,69.09,19
4,54C0631E-D2AD-EC11-9064-005056A7A8DF,12853,32931,1.0,12,49.09,16


In [136]:
df_checks_good_milk = df_checks_short[(df_checks_short['id_tt_cl'].isin(lis_good_shops)) & (df_checks_short['id_tov_cl'].isin(milk_items_by_id)) ]

In [137]:
df_checks_bad_milk = df_checks_short[(df_checks_short['id_tt_cl'].isin(lis_bad_shops)) & (df_checks_short['id_tov_cl'].isin(milk_items_by_id)) ]

In [221]:
df_checks_good_milk_it_sum = df_checks_good_milk.groupby(['id_tov_cl']).agg({'BaseSum': ['sum']}).reset_index()
#Исправляем табличку 
df_checks_good_milk_it_sum.columns = list(map(''.join, df_checks_good_milk_it_sum.columns.values))
df_checks_good_milk_it_sum['BaseSumsum'] = df_checks_good_milk_it_sum['BaseSumsum'].astype('int')

In [222]:
df_checks_bad_milk_it_sum = df_checks_bad_milk.groupby(['id_tov_cl']).agg({'BaseSum': ['sum']}).reset_index()
#Исправляем табличку 
df_checks_bad_milk_it_sum.columns = list(map(''.join, df_checks_bad_milk_it_sum.columns.values))
df_checks_bad_milk_it_sum['BaseSumsum'] = df_checks_bad_milk_it_sum['BaseSumsum'].astype('int')


In [223]:
df_OOS4 = df_OOS3.merge(df_checks_bad_milk_it_sum[['BaseSumsum','id_tov_cl']], on='id_tov_cl', how='left') 
df_OOS4 = df_OOS4.rename(columns={'BaseSumsum': 'Bad_sum'})


In [248]:
######## Вычисляем на сколько нужно нормировать продажи 

df_worst_milk_sales_bad_shops = pd.read_pickle("./df_worst_milk_sales_bad_shops.pkl")
df_best_milk_sales_good_shops = pd.read_pickle("./df_best_milk_sales_good_shops.pkl")
########


lis_good_shops = df_best_milk_sales_good_shops.index.values.tolist()
lis_bad_shops = df_worst_milk_sales_bad_shops.index.values.tolist()

good_shops_overal_sales = df_checks_short['BaseSum'].sum()

########
good_shops_overal_sales = df_checks_short['BaseSum'][df_checks_short['id_tt_cl'].isin(lis_good_shops)].sum()
bad_shops_overal_sales = df_checks_short['BaseSum'][df_checks_short['id_tt_cl'].isin(lis_bad_shops)].sum()
coeff_good_bad = bad_shops_overal_sales/good_shops_overal_sales

########
coeff_good_bad_r = np.round(coeff_good_bad, 3)


In [249]:
df_OOS5 = df_OOS4.merge(df_checks_good_milk_it_sum[['BaseSumsum','id_tov_cl']], on='id_tov_cl', how='left') 
df_OOS5 = df = df_OOS5.rename(columns={'BaseSumsum': 'Good_sum'})
df_OOS5['Good_sum'] = df_OOS5['Good_sum']*coeff_good_bad_r

In [250]:
df_OOS5['Sum_diff'] = df_OOS5['Good_sum']/df_OOS5['Bad_sum']

In [251]:
df_OOS5.sort_values('OOS_percent', ascending=False).to_excel('df_OOS5.xlsx')

In [252]:
df_OOS5.sort_values('OOS_percent', ascending=False).to_pickle('df_OOS5.pkl')